In [ ]:
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable 
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
channels = 1 
img_size = 28 
img_shape = (channels, img_size, img_size) 

In [ ]:
latent_dim = 100 

In [ ]:
cuda = True if torch.cuda.is_available() else False 

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        def block(input_features, output_features, normalize=True):
            layers = [nn.Linear(input_features, output_features)]
            if normalize: 
                layers.append(nn.BatchNorm1d(output_features, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True)) 
            return layers 
        
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False), 
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))), 
            nn.Tanh() 
        )

    def forward(self, z): 
        img = self.model(z) 
        img = img.view(img.size(0), *img_shape) 
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512), 
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid() 
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1) 
        validity = self.model(img_flat) 
        return validity

In [ ]:
adversarial_loss = torch.nn.BCELoss()

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
generator

In [ ]:
discriminator

In [ ]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

In [ ]:
class DatasetMNIST(Dataset): 
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
           
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28,28,1))
        label = self.data.iloc[index, 0]
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label
        

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
train

In [ ]:
for index in range(1, 6): # N : 5 (Number of Image)
    temp_image = train.iloc[index, 1:].values.astype(np.uint8).reshape((28,28,1))
    temp_label = train.iloc[index, 0]
    print('Shape of Image : ',temp_image.shape)
    print('label : ', temp_label)

In [ ]:
dataset = DatasetMNIST(file_path='../input/digit-recognizer/train.csv', 
                       transform=transforms.Compose(
                           [transforms.ToTensor(), # ToTensor() : np.array (H, W, C) -> tensor (C, H, W)
                            transforms.Normalize([0.5],[0.5])]
                       ))

In [ ]:
temp_img, _ =  dataset.__getitem__(0) 

In [ ]:
temp_img.size() 

In [ ]:
temp_img.max(), temp_img.min() 

In [ ]:
batch_size = 64 
dataloader = DataLoader( 
    dataset,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
temp_images, _ = iter(dataloader).next() 
print('images shape on batch size = {}'.format(temp_images.size()))

In [ ]:
b1 = 0.5
b2 = 0.999
lr = 0.0002 

In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1,b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1,b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
n_epochs = 10 
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(tqdm(dataloader)): 
        
        
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False) 
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False) 
                                                                                
        
        real_imgs = imgs.type(Tensor) 
     
        optimizer_G.zero_grad()
        
        z = Tensor(np.random.normal(0, 1, (imgs.shape[0],latent_dim))) 
   
        
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid) 
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake) 
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        sample_z_in_train = Tensor(np.random.normal(0, 1, (imgs.shape[0],latent_dim)))
     
        sample_gen_imgs_in_train = generator(sample_z_in_train).detach().cpu()
        if ((i+1) % 200) == 0: 
            nrow=1
            ncols=5
            fig, axes = plt.subplots(nrows=nrow,ncols=ncols, figsize=(8,2))
            plt.suptitle('EPOCH : {} | BATCH(ITERATION) : {}'.format(epoch+1, i+1))
            for ncol in range(ncols):
                axes[ncol].imshow(sample_gen_imgs_in_train.permute(0,2,3,1)[ncol], cmap='gray')
                axes[ncol].axis('off')
            plt.show()
    print(
        "[Epoch: %d/%d] [Batch: %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch+1, n_epochs, i+1, len(dataloader), d_loss.item(), g_loss.item())
    )

In [ ]:
count = 1
for i, (imgs,label) in enumerate(dataloader):
    print('Shape of Batch Images : \n', imgs.shape)
    print('Labels (1~64) : \n', label)
    print('-'*100)
    if count == 5:
        break
    else:
        count += 1

In [ ]:
Tensor(10,1) 

In [ ]:
Tensor(10,1).fill_(1.0) 

In [ ]:
sample_img = iter(dataloader).next()[0]

In [ ]:
sample_img.shape, sample_img.dtype

In [ ]:
sample_img.requires_grad 

In [ ]:
Variable(sample_img).requires_grad 

In [ ]:
sample_img.requires_grad_(True) 

In [ ]:
sample_img.requires_grad 

In [ ]:
np.random.normal(0,1,(64,100))

In [ ]:
np.random.normal(0,1,(64,100)).shape

In [ ]:
sample_z = Tensor(np.random.normal(0, 1, (64,100)))
sample_z.shape

In [ ]:
sample_gen_imgs = generator(sample_z)
sample_gen_imgs.shape

In [ ]:
sample_discrim_result = discriminator(sample_gen_imgs)
sample_discrim_result.shape

In [ ]:
adversarial_loss

In [ ]:
sample_valid.shape

In [ ]:
sample_valid = Tensor(64,1).fill_(1.0)
sample_g_loss = adversarial_loss(sample_discrim_result, sample_valid)
sample_g_loss

In [ ]:
!pip install torchviz
import torchviz

In [ ]:
X = torch.ones((28,28), dtype=torch.float32, requires_grad=True)
square_X = X**2
cubic_X = X**3

result = (square_X+cubic_X).sum()

torchviz.make_dot(result)


In [ ]:
X = torch.ones((28,28), dtype=torch.float32, requires_grad=True)
square_X = X**2
cubic_X = X.detach()**3

result = (square_X+cubic_X).sum()

torchviz.make_dot(result)